In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm, gmean

In [2]:
r = 0.01
sigma = 0.3
K = 100
S0 = 110
T = 1
m = 12

# Asian Option Pricing on the Arithmetic Average

In [3]:
def AsianOption_no_CV(r, sigma, K, S0, T, m, I):
    dt = T / m
    path = np.zeros((m + 1, I))    
    path[0] = S0
    
    # Generate stock prices
    for t in range(1, m + 1):
        path[t] = path[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt 
                + sigma * np.sqrt(dt) * np.random.standard_normal(I))
    
    # Calcualte the mean of each column
    path = np.delete(path, 0, axis = 0)
    ari_mean = path.mean(axis = 0)

    # Calculate Asian Option price of each column
    AsianOption_column = []
    for i in range(0, I):
        AsianOption_column.append(np.exp(-r * T) * np.maximum(ari_mean[i] - K, 0))
    
    # Calculate the mean of Asian option price
    AsianOption = np.mean(AsianOption_column)
    
    # Calculate error
    diff = 0
    for b in range(0, I):
        diff += (AsianOption_column[b] - AsianOption) ** 2
    Error = np.sqrt(diff / ((I - 1) * I))
    
    return AsianOption, Error
    return path

In [4]:
AsianOption_no_CV_List = []
AsianOption_no_CV_Error_List = []

sim_num = [1000, 10000, 100000, 1000000, 10000000]

for I in sim_num:
    
    AsianOption_no_CV_List.append(AsianOption_no_CV(r, sigma, K, S0, T, m, I)[0])
    AsianOption_no_CV_Error_List.append(AsianOption_no_CV(r, sigma, K, S0, T, m, I)[1])

df = pd.DataFrame({'Price': AsianOption_no_CV_List, 'Error': AsianOption_no_CV_Error_List}, 
                  index = ['1000', '10000', '100000', '1000000', '10000000'])
df

,Price,Error
1000,15.803724,0.543060
10000,13.839568,0.165359
100000,13.946712,0.052332
1000000,13.981316,0.016585
10000000,13.952951,0.005247


In the basic algorithm for computing the price of the Asian option on the arithmetic average, when I increase the number of simulations from 1000 to 10000000, the price of Asian option converges to 13.95 and the estimated error becomes smaller and smaller, which indicates the estimated price is more and more accurate.

# Asian Option Pricing that Incorporates the Control Variates Variance Reduction Technique

In [5]:
# Calculate the price of asian option that incorporates the control variates variance reduction technique
def AsianOption_CV(r, sigma, K, S0, T, m, I):
    # Simulate stock price
    dt = T / m
    path = np.zeros((m + 1, I))    
    path[0] = S0
    
    for t in range(1, m + 1):
        path[t] = path[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt 
                + sigma * np.sqrt(dt) * np.random.standard_normal(I))
    
    path = np.delete(path, 0, axis = 0)
    
    # Calculate X with geometric mean
    geo_mean = gmean(path, axis = 0)
    X = []
    
    for i in range(0, I):
        X.append(np.exp(-r * T) * np.maximum(geo_mean[i] - K, 0))
        
    geo_opt_price  = np.mean(X)
    
    # Calculate Y with arithmetic mean
    ari_mean = path.mean(axis = 0)
    Y = []
    
    for j in range(0, I):
        Y.append(np.exp(-r * T) * np.maximum(ari_mean[j] - K, 0))
        
    ari_opt_price  = np.mean(Y)
    
    # Calculate B
    numerator = []
    denominator = []
    
    for a in range(0, I):
        numerator.append((Y[a] - ari_opt_price) * (X[a] - geo_opt_price))
        denominator.append((X[a] - geo_opt_price) ** 2)

    B = np.sum(numerator) / np.sum(denominator)
    
    # Calculate E[X] which is known in closed form     
    T_bar = T * (m + 1)/ (2 * m)
    sum_t = 0
    for b in range(1, m + 1):
        sum_t += (2 * b - 1) * ((m + 1 - b) / m)
    variance_bar = (sigma ** 2 * sum_t)/(T_bar * m ** 2)
    delta = 0.5 * sigma ** 2 - 0.5 * variance_bar
    d1 = (np.log(S0 / K) + (r - delta + 0.5 * variance_bar) * T_bar) / np.sqrt(variance_bar * T_bar)
    d2 = d1 - np.sqrt(variance_bar * T_bar)
    Geo_AsianOptionPrice = np.exp(-delta * T_bar - r * (T - T_bar)) * S0 * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)
    
    # Calculate the final result
    AsianOption = [] 
    for c in range (0, I):
        AsianOption.append(Y[c] - B * (X[c] - Geo_AsianOptionPrice))
    AsianOption_CV = np.mean(AsianOption)
    
    # Correlation coefficient beweteen X and Y
    correlation = np.corrcoef(X, Y)
    
    # Calculate error
    diff = 0
    
    for d in range(0, I):
        diff += (AsianOption[d] - AsianOption_CV) ** 2
    
    Error = np.sqrt(diff / ((I - 1) * I))
    
    return AsianOption_CV, correlation[0,1], Error

In [6]:
AsianOption_CV_List = []
AsianOption_CV_Corr_List = []
AsianOption_CV_Error_List = []

sim_num = [1000, 10000, 100000, 1000000, 10000000]

for I in sim_num:
    
    AsianOption_CV_List.append(AsianOption_CV(r, sigma, K, S0, T, m, I)[0])
    AsianOption_CV_Corr_List.append(AsianOption_CV(r, sigma, K, S0, T, m, I)[1])
    AsianOption_CV_Error_List.append(AsianOption_CV(r, sigma, K, S0, T, m, I)[2])

df = pd.DataFrame({'Price': AsianOption_CV_List, 'Corr': AsianOption_CV_Corr_List,
                   'Error': AsianOption_CV_Error_List}, 
                  index = ['1000', '10000', '100000', '1000000', '10000000'])
df

,Price,Corr,Error
1000,13.950803,0.999440,0.021663
10000,13.954477,0.999356,0.006004
100000,13.952449,0.999352,0.001887
1000000,13.951096,0.999360,0.000593
10000000,13.951571,0.999361,0.000188


In the more sophisticated algorithm that incorporates the control variates variance reduction technique, when I increase the number of simulations from 1000 to 10000000, the price of Asian option also converges to 13.95, same as the first algorithm. But it converges to 13.95 more quickly, it is obvious that when the number of simulations is 1000, the estimated price is about 13.95, meanwhile the estimated error becomes smaller and smaller more quickly. Besides, the correlation coefficients which measures the speedup due to the variance reduction technique are greater than 0.99 for different simulation amount. 

Thus, the second algorithm works more efficiently and effectively than the first algorithm, the accuracy is better for the second algorithm than the first one.